In [1]:
from termcolor import colored
import os 
from PyPDF2 import PdfReader
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback
from termcolor import colored
os.environ["OPENAI_API_KEY"] = #INSERT YOUR KEY HERE#
def read_PDF(pdf_dir):
    doc_reader = PdfReader(pdf_dir)
    raw_text = ''
    for i, page in enumerate(doc_reader.pages):
        text = page.extract_text()
        if text:
            raw_text += text
    return raw_text

# Splits raw text into smaller chunks with overlap
def split_text(raw_text):
    text_splitter = CharacterTextSplitter(        
        separator = "\n",
        chunk_size = 2500,
        chunk_overlap  = 400, #striding over the text
        length_function = len,
    )
    texts = text_splitter.split_text(raw_text)
    return texts

directory = r"C:\Users\felip\OneDrive - Fundacao Getulio Vargas - FGV\!_Paper04_Chat_PDF\TCE_PIAUI\VOTREL"
files = os.listdir(directory)
pdfs = [file for file in files if file.lower().endswith('.pdf')]
print(len(pdfs),' PDFs were found!\n\n')

732  PDFs were found!




In [25]:
full_text = ''
for k in pdfs[0:10]:
    full_text = full_text + read_PDF(directory+'\\'+k)

In [26]:
len(full_text)

240018

In [27]:
prompt_pattern = """
Your Role: An auditor expert in finding patterns. 
Your Task: identify patterns in the accountability documents of various municipalities.
Using only information from these context: {context}
Question: {question}
Do not add external information on the replay and do not include opinions.
The patters found are:
"""
question = "Identify patterns on the municipalities accountability:"

prompt_pattern_extraction = PromptTemplate(template=prompt_pattern, input_variables=['context', 'question'])

def cost_and_tokens(usage_list):  
    total_cost = 0
    total_tokens = 0
    for c in usage_list:
        total_cost += c.total_cost
        total_tokens += c.total_tokens
    return total_cost,total_tokens

def QA_LLM(query,prompt_input):
    
    QA = RetrievalQA.from_chain_type(llm=OpenAI(model_name='gpt-3.5-turbo',#cost of $0.0015 / 1K tokens
                                                temperature = 0), 
                                      chain_type="stuff", 
                                      retriever=retriever,
                                      #verbose=True,
                                      chain_type_kwargs={"prompt": prompt_input},
                                      return_source_documents=True)
    with get_openai_callback() as cost:
        answer = QA(query,prompt_input)     
    return answer, cost

In [28]:
embeddings = OpenAIEmbeddings()
texts = split_text(full_text)
docsearch = FAISS.from_texts(texts, embeddings)
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":4})
pattern, pattern_cost = QA_LLM(question,prompt_pattern_extraction) 

C:\Users\felip\anaconda3\lib\site-packages\langchain\llms\openai.py:179: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
C:\Users\felip\anaconda3\lib\site-packages\langchain\llms\openai.py:753: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [24]:
pattern['result']

'1. Lack of compliance with transparency requirements: In all three municipalities (Acauã, Agricolândia, and Água Branca), there were inconsistencies and deficiencies in the information provided on the transparency portals. This includes missing information on employee registration and CPF numbers, lack of information on revenues and expenses, absence of reports and documents related to financial management, and incomplete organizational information.\n\n2. Non-compliance with legal and regulatory requirements: The municipalities failed to meet the minimum requirements established by the regulatory norms. This includes the absence of information on employee registration and CPF numbers, missing reports on financial management, lack of information on salary plans and administrative organization, and incomplete disclosure of addresses and phone numbers.\n\n3. Inadequate financial management: The municipalities showed deficiencies in financial management, including the invalidity of the Ce

In [29]:
pattern['result']

"1. Non-compliance with education development goals: The municipalities analyzed did not meet the projected targets for the Index of Basic Education Development (IDEB) in the final years of primary education. This pattern was observed in multiple years, indicating a consistent issue in educational performance.\n\n2. Lack of transparency in municipal portals: The municipalities' transparency portals were found to have various inconsistencies and deficiencies. These included the absence of information on enrollment, CPF (Brazilian identification number), and benefits, as well as the lack of information on bidding results, contracts, and agreements. Additionally, the portals did not provide access to local legislation, salary plans, or organizational structure. The absence of financial reports and the inability to save reports in open formats for analysis were also noted. Furthermore, there was a lack of information on the possibility of submitting access requests in person and the absenc